This code run in the Kaggle notebook environment for checking the Bayesian linear regression, which is based on CARLOS SOUZA.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pymc3 as pm

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
df_train = pd.read_csv('C:/Users/u1363606/Downloads/OSIC/train.csv')
df_test = pd.read_csv('C:/Users/u1363606/Downloads/OSIC/test.csv')

In [3]:
#label encoder for patientid
le_id = LabelEncoder()
df_train['PatientID'] = le_id.fit_transform(df_train['Patient'])

# the log value of FVC is more approporiate in this case
df_train['FVC_base'] = df_train['FVC']
df_train['FVC'] = np.log(df_train['FVC'])

df_train.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,PatientID,FVC_base
0,ID00007637202177411956430,-4,7.747165,58.253649,79,Male,Ex-smoker,0,2315
1,ID00007637202177411956430,5,7.702556,55.712129,79,Male,Ex-smoker,0,2214
2,ID00007637202177411956430,7,7.630947,51.862104,79,Male,Ex-smoker,0,2061
3,ID00007637202177411956430,9,7.670429,53.950679,79,Male,Ex-smoker,0,2144
4,ID00007637202177411956430,11,7.634821,52.063412,79,Male,Ex-smoker,0,2069


In [4]:
#One-Hot encode for category features:sex and smokingstatus
COLS = ['Sex','SmokingStatus']
for col in COLS:
    for mod in df_train[col].unique():
        df_train[mod] = (df_train[col] == mod).astype(int)

In [5]:
df_train

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,PatientID,FVC_base,Male,Female,Ex-smoker,Never smoked,Currently smokes
0,ID00007637202177411956430,-4,7.747165,58.253649,79,Male,Ex-smoker,0,2315,1,0,1,0,0
1,ID00007637202177411956430,5,7.702556,55.712129,79,Male,Ex-smoker,0,2214,1,0,1,0,0
2,ID00007637202177411956430,7,7.630947,51.862104,79,Male,Ex-smoker,0,2061,1,0,1,0,0
3,ID00007637202177411956430,9,7.670429,53.950679,79,Male,Ex-smoker,0,2144,1,0,1,0,0
4,ID00007637202177411956430,11,7.634821,52.063412,79,Male,Ex-smoker,0,2069,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,7.905442,66.594637,73,Male,Never smoked,175,2712,1,0,0,1,0
1545,ID00426637202313170790466,19,7.999007,73.126412,73,Male,Never smoked,175,2978,1,0,0,1,0
1546,ID00426637202313170790466,31,7.975221,71.407524,73,Male,Never smoked,175,2908,1,0,0,1,0
1547,ID00426637202313170790466,43,7.997999,73.052745,73,Male,Never smoked,175,2975,1,0,0,1,0


In [6]:
df1 = df_train[['Patient', 'Weeks', 'Percent']].sort_values(by=['Patient', 'Weeks'])
df1 = df_train.groupby('Patient').head(1)
df1 = df1.rename(columns={'Percent': 'Percent_base'})
df_train = pd.merge(df_train, df1[['Patient', 'Percent_base']], how='left',on='Patient')
df_train

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,PatientID,FVC_base,Male,Female,Ex-smoker,Never smoked,Currently smokes,Percent_base
0,ID00007637202177411956430,-4,7.747165,58.253649,79,Male,Ex-smoker,0,2315,1,0,1,0,0,58.253649
1,ID00007637202177411956430,5,7.702556,55.712129,79,Male,Ex-smoker,0,2214,1,0,1,0,0,58.253649
2,ID00007637202177411956430,7,7.630947,51.862104,79,Male,Ex-smoker,0,2061,1,0,1,0,0,58.253649
3,ID00007637202177411956430,9,7.670429,53.950679,79,Male,Ex-smoker,0,2144,1,0,1,0,0,58.253649
4,ID00007637202177411956430,11,7.634821,52.063412,79,Male,Ex-smoker,0,2069,1,0,1,0,0,58.253649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,7.905442,66.594637,73,Male,Never smoked,175,2712,1,0,0,1,0,71.824968
1545,ID00426637202313170790466,19,7.999007,73.126412,73,Male,Never smoked,175,2978,1,0,0,1,0,71.824968
1546,ID00426637202313170790466,31,7.975221,71.407524,73,Male,Never smoked,175,2908,1,0,0,1,0,71.824968
1547,ID00426637202313170790466,43,7.997999,73.052745,73,Male,Never smoked,175,2975,1,0,0,1,0,71.824968


In [7]:
# get dummies for Age,if age <50, the paitent is young patient
df_train.loc[df_train['Age'] < 50, 'Young_Patient'] = 1
df_train.loc[df_train['Age'] >= 50, 'Young_Patient'] = 0
df_train['Young_Patient'] = pd.get_dummies(df_train['Young_Patient'])
df_train.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,PatientID,FVC_base,Male,Female,Ex-smoker,Never smoked,Currently smokes,Percent_base,Young_Patient
0,ID00007637202177411956430,-4,7.747165,58.253649,79,Male,Ex-smoker,0,2315,1,0,1,0,0,58.253649,1
1,ID00007637202177411956430,5,7.702556,55.712129,79,Male,Ex-smoker,0,2214,1,0,1,0,0,58.253649,1
2,ID00007637202177411956430,7,7.630947,51.862104,79,Male,Ex-smoker,0,2061,1,0,1,0,0,58.253649,1
3,ID00007637202177411956430,9,7.670429,53.950679,79,Male,Ex-smoker,0,2144,1,0,1,0,0,58.253649,1
4,ID00007637202177411956430,11,7.634821,52.063412,79,Male,Ex-smoker,0,2069,1,0,1,0,0,58.253649,1


Our model can be describes as:
$$
FVC_{ij} \sim \mathcal{N}(\alpha_i + X_{ij} \beta_i, \sigma)
$$

Reference: Carlos Souza's Bayesian notebook

​
$X_{ij}$ is a vector of the patient *i* at timestep *j* :Male ;Ex-smoker;Currently-smokes;Percent from patient *i* at baseline moment;The week 


prior distributions for the parameters

In [9]:
# calculate priors for these variables
FVC_smokers_mean = df_train.loc[df_train['SmokingStatus'] == 'Currently smokes', 'FVC'].mean()
FVC_smokers_stdev = df_train.loc[df_train['SmokingStatus'] == 'Currently smokes', 'FVC'].std() / 2

FVC_smokers_beta_mean = df_train.loc[(df_train['SmokingStatus'] == 'Currently smokes') & (df_train['Weeks'] > 70), 'FVC'].mean() - df_train.loc[(df_train['SmokingStatus'] == 'Currently smokes') & (df_train['Weeks'] < 10), 'FVC'].mean()
FVC_smokers_beta_weeks_mean = FVC_smokers_beta_mean / 70
FVC_smokers_beta_others_sd = 1

In [10]:
# update priors from the observed data
smokers_n_patients = df_train.loc[df_train['Currently smokes'] == 1, 'Patient'].nunique()
smokers_FVC_obs = df_train.loc[df_train['Currently smokes'] == 1, 'FVC'].values
smokers_PatientID = df_train.loc[df_train['Currently smokes'] == 1, 'PatientID'].values

smokers_Weeks = df_train.loc[df_train['Currently smokes'] == 1, 'Weeks'].values
smokers_Male = df_train.loc[df_train['Currently smokes'] == 1, 'Male'].values
smokers_Young =df_train.loc[df_train['Currently smokes'] == 1, 'Young_Patient'].values
smokers_Percent = df_train.loc[df_train['Currently smokes'] == 1, 'Percent_base'].values

In [ ]:
with pm.Model() as model_smokers:
    
    FVC_obs_shared = pm.Data("FVC_obs_shared", smokers_FVC_obs)
    PatientID_shared = pm.Data('PatientID_shared', smokers_PatientID)
    Weeks_shared = pm.Data('Weeks_shared', smokers_Weeks)
    Male_shared = pm.Data('Male_shared', smokers_Male)
    YoungPatient_shared = pm.Data('YoungPatient_shared', smokers_Young)
    Percent_shared = pm.Data('Percent_shared', smokers_Percent)
    # Model parameter priors
    a = pm.Normal('a', mu = FVC_smokers_mean, sigma = FVC_smokers_stdev)
    b_week = pm.Normal('b_week', mu = FVC_smokers_beta_weeks_mean, sigma = 1.)
    b_young = pm.Normal('b_young', mu = 0, sigma = FVC_smokers_beta_others_sd)
    b_weeks_young = pm.Normal('b_weeks_young', mu = 0, sigma = FVC_smokers_beta_others_sd)
    b_gender = pm.Normal('b_gender', mu = 0, sigma = FVC_smokers_beta_others_sd)
    b_weeks_gender = pm.Normal('b_weeks_gender', mu = 0, sigma = FVC_smokers_beta_others_sd)
    b_weeks_young_gender = pm.Normal('b_weeks_young_gender', mu = 0, sigma = FVC_smokers_beta_others_sd)
    b_percent = pm.Normal('b_percent', mu = 0, sigma = FVC_smokers_beta_others_sd)
    # Model error
    sigma = pm.HalfNormal('sigma', np.log(70.))
    # Model formula
    FVC_est = a + b_week * Weeks_shared + b_young * YoungPatient_shared + b_weeks_young * Weeks_shared * YoungPatient_shared + b_gender * Male_shared + b_weeks_gender * Weeks_shared * Male_shared + b_weeks_young_gender * Weeks_shared * YoungPatient_shared * Male_shared + b_percent * Percent_shared
    # Data likelihood
    FVC_like = pm.Normal('FVC_like', mu = FVC_est, sigma = sigma, observed = FVC_obs_shared)

In [16]:
with model_smokers:
    trace_smokers = pm.sample(2000, tune = 15000, target_accept = .95, init = "adapt_diag")

In [17]:
with model_smokers:
    pm.traceplot(trace_smokers);

In [34]:
aux_train = df_train.groupby('Patient').first().reset_index()
pred_template_train = []
for i in range(df_train['Patient'].nunique()):
    train = pd.DataFrame(columns=['PatientID', 'Weeks'])
    train['Weeks'] = np.arange(-12, 134)
    train['PatientID'] = i
    train['Male'] = aux_train[aux_train['PatientID'] == i]['Male'].values[0]
    train['Currently smokes'] = aux_train[aux_train['PatientID'] == i]['Currently smokes'].values[0]
    train['Young_Patient'] = aux_train[aux_train['PatientID'] == i]['Young_Patient'].values[0]
    train['Percent_base'] = aux_train[aux_train['PatientID'] == i]['Percent_base'].values[0]
    pred_template_train.append(df_train)
pred_template_train = pd.concat(pred_template_train, ignore_index=True)

In [35]:
pred_template_train

In [20]:
with model_smokers:
    pm.set_data({
        "PatientID_shared": pred_template_train.loc[pred_template_train['Currently smokes'] == 1, 'PatientID'].values.astype(int),
        "Weeks_shared": pred_template_train.loc[pred_template_train['Currently smokes'] == 1, 'Weeks'].values.astype(int),
        "Male_shared": pred_template_train.loc[pred_template_train['Currently smokes'] == 1, 'Male'].values.astype(int),
        "YoungPatient_shared": pred_template_train.loc[pred_template_train['Currently smokes'] == 1, 'Young_Patient'].values.astype(int),
        "Percent_shared": pred_template_train.loc[pred_template_train['Currently smokes'] == 1, 'Percent_base'].values,
        "FVC_obs_shared": np.zeros(len(pred_template_train.loc[pred_template_train['Currently smokes'] == 1, :])).astype(int),
    })
    post_pred_smokers_train = pm.sample_posterior_predictive(trace_smokers)

In [25]:
post_pred_smokers_train